In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
import numpy as np
from keras.applications.vgg16 import preprocess_input

## Training Set DIR

In [2]:
train_dataset_path='D:/DatasetMedicalWasteCropped/'

## Testing Set DIR

In [3]:
test_indoor_dataset_path='D:/DatasetMedicalWasteTestLabeledCropped/indoor'
test_outdoor_dataset_path='D:/DatasetMedicalWasteTestLabeledCropped/outdoor'
test_belt_dataset_path='D:/DatasetMedicalWasteTestLabeledCropped/belt'

In [4]:
img_height=299
img_width=299
batch_size=32

In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_dataset_path,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3868 files belonging to 41 classes.
Using 3095 files for training.


In [6]:
class_names = train_ds.class_names
class_number = len(class_names)
print(class_names)
print(f'number of class = {class_number}')

['1WayConnectorforFoley', '2WayConnectorforFoley', '2WayFoleyCatheter', '3WayConnectorforFoley', '3Waystopcock', 'AlcoholBottle', 'AlcoholPad', 'BootCover', 'CottonBall', 'CottonSwap', 'Dilator', 'DisposableInfusionSet', 'ExtensionTube', 'FaceShield', 'FrontLoadSyringe', 'GauzePad', 'Glove', 'GuideWire', 'LiquidBottle', 'Mask', 'NGTube', 'NasalCannula', 'Needle', 'OxygenMask', 'PPESuit', 'PharmaceuticalProduct', 'Pill', 'PillBottle', 'PrefilledHumidifier', 'PressureConnectingTube', 'ReusableHumidifier', 'SodiumChlorideBag', 'SterileHumidifierAdapter', 'SurgicalBlade', 'SurgicalCap', 'SurgicalSuit', 'Syringe', 'TrachealTube', 'UrineBag', 'Vaccinebottle', 'WingedInfusionSet']
number of class = 41


In [7]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  train_dataset_path,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3868 files belonging to 41 classes.
Using 773 files for validation.


In [12]:
test_indoor_ds = tf.keras.utils.image_dataset_from_directory(
  test_indoor_dataset_path,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1794 files belonging to 41 classes.


In [13]:
test_outdoor_ds = tf.keras.utils.image_dataset_from_directory(
  test_outdoor_dataset_path,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3140 files belonging to 41 classes.


In [14]:
test_belt_ds = tf.keras.utils.image_dataset_from_directory(
  test_belt_dataset_path,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 2206 files belonging to 41 classes.


In [8]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

In [9]:
def resize_and_rescale(image):
  image = tf.cast(image, tf.float32)
  '''image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])'''
  image = (image / 255.0)
  return image

In [10]:
#Apply the preprocessing layers to your dataset
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

def prepare(ds, shuffle=False, augment=False):
  # Resize and rescale all datasets.
  ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

  if shuffle:
    ds = ds.shuffle(1000)

  # Batch all datasets.
  #ds = ds.batch(batch_size)

  # Use data augmentation only on the training set.
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

  # Use buffered prefetching on all datasets.
  return ds.prefetch(buffer_size=AUTOTUNE)

In [15]:
train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)
test_indoor_ds = prepare(test_indoor_ds)
test_outdoor_ds = prepare(test_outdoor_ds)
test_belt_ds = prepare(test_belt_ds)

In [ ]:
#Creating generator for Test Indoor DataSet
'''test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,rescale=1./255,)
test_indoor_generator = test_datagen.flow_from_directory(
        test_indoor_dataset_path,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')'''

In [ ]:
#Creating generator for Test Outdoor DataSet
'''test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,rescale=1./255,)
test_outdoor_generator = test_datagen.flow_from_directory(
        test_outdoor_dataset_path,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')'''

In [ ]:
#Creating generator for Test On Belt DataSet
'''test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,rescale=1./255,)
test_belt_generator = test_datagen.flow_from_directory(
        test_belt_dataset_path,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical')'''

# Build and Train CNN (VGG16) -> https://www.kaggle.com/aditya276/bird-cnn-vgg16-99-accuracy-on-test-set#3.)-Evaluation-on-Test-Set

In [113]:
base_model=keras.applications.VGG16(
    input_shape=(img_height, img_width,3),
    include_top=False,
    classes=class_number,
    weights="imagenet",
    )

In [129]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [130]:
model = tf.keras.Sequential([
  base_model,
  layers.Flatten(),
  tf.keras.layers.Dense(1000, activation='relu'),
  tf.keras.layers.Dense(class_number),
])

In [126]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [112]:
#freeze the base model
base_model.trainable = False

In [103]:
#Create new model on top
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
model=Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(2048,activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(0.35))
model.add(Dense(2048,activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(0.35))
model.add(Dense(200,activation='softmax',kernel_initializer='glorot_normal'))

In [133]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_19 (Dense)             (None, 1000)              25089000  
_________________________________________________________________
dense_20 (Dense)             (None, 41)                41041     
Total params: 39,844,729
Trainable params: 39,844,729
Non-trainable params: 0
_________________________________________________________________


In [135]:
#Train the model on new data.
model.compile(optimizer=keras.optimizers.Adam(1e-4),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
history=model.fit(train_ds,validation_data=val_ds,epochs=100,workers=10,use_multiprocessing=True)

Epoch 1/100
97/97 [==============================] - 29s 276ms/step - loss: 10.5297 - accuracy: 0.0061 - val_loss: 10.3918 - val_accuracy: 0.0039
Epoch 2/100
97/97 [==============================] - 28s 277ms/step - loss: 10.5297 - accuracy: 0.0061 - val_loss: 10.3918 - val_accuracy: 0.0039
Epoch 3/100
97/97 [==============================] - 28s 277ms/step - loss: 10.5297 - accuracy: 0.0061 - val_loss: 10.3918 - val_accuracy: 0.0039
Epoch 4/100
97/97 [==============================] - 27s 276ms/step - loss: 10.5297 - accuracy: 0.0061 - val_loss: 10.3918 - val_accuracy: 0.0039
Epoch 5/100
97/97 [==============================] - 27s 276ms/step - loss: 10.5297 - accuracy: 0.0061 - val_loss: 10.3918 - val_accuracy: 0.0039
Epoch 6/100
12/97 [==>...........................] - ETA: 21s - loss: 10.3822 - accuracy: 0.0078

KeyboardInterrupt: 

In [ ]:
#Some visualizations
import matplotlib.pyplot as plt
#Loss
plt.plot(history.history['loss'],label='loss')
plt.plot(history.history['val_loss'],label='val_loss')
plt.legend()
plt.show()
#Accuracy
plt.plot(history.history['accuracy'],label='acc')
plt.plot(history.history['val_accuracy'],label='val_acc')
plt.legend()
plt.show()

# TensorFlow - Hub Inception-v3

In [16]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 299, 299, 3)
(32,)


In [17]:
import tensorflow_hub as hub

In [18]:
inception_v3 = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"
feature_extractor_model  = inception_v3
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(299, 299, 3),
    trainable=False)

In [19]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

(32, 1001)


In [20]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

In [21]:
predictions = model(image_batch)

In [22]:
predictions.shape

TensorShape([32, 41])

In [24]:
import time
import datetime
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1) # Enable histogram computation for every epoch.

In [25]:
NUM_EPOCHS = 10

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)

Epoch 1/10
97/97 [==============================] - 29s 182ms/step - loss: 3.1652 - acc: 0.2220 - val_loss: 3.0425 - val_acc: 0.2329
Epoch 2/10
97/97 [==============================] - 18s 108ms/step - loss: 2.9396 - acc: 0.2475 - val_loss: 3.0177 - val_acc: 0.2238
Epoch 3/10
97/97 [==============================] - 18s 109ms/step - loss: 2.8310 - acc: 0.2779 - val_loss: 2.8758 - val_acc: 0.2794
Epoch 4/10
97/97 [==============================] - 18s 110ms/step - loss: 2.7592 - acc: 0.2869 - val_loss: 2.8180 - val_acc: 0.2820
Epoch 5/10
97/97 [==============================] - 18s 108ms/step - loss: 2.7097 - acc: 0.2856 - val_loss: 2.8508 - val_acc: 0.2846
Epoch 6/10
97/97 [==============================] - 18s 108ms/step - loss: 2.6661 - acc: 0.2985 - val_loss: 2.8020 - val_acc: 0.2833
Epoch 7/10
97/97 [==============================] - 18s 102ms/step - loss: 2.6399 - acc: 0.3147 - val_loss: 2.7289 - val_acc: 0.2872
Epoch 8/10
97/97 [==============================] - 18s 105ms/step - 

# Fine-Tuning

In [ ]:
base_model=model.layers[0]

In [ ]:
#Un-Freezing last 2 blocks(i.e. block4 and 5)

base_model.trainable = True

set_trainable = False
for layer in base_model.layers:
    if layer.name == 'block4_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
        
base_model.summary()
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(1e-5),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#Lets fine-tune finally....
history=model.fit(train_generator,epochs=30,validation_data=val_generator,workers=10,use_multiprocessing=True)

In [ ]:
#few more epochs with low l_rate
model.compile(optimizer=keras.optimizers.Adam(1e-6),loss='categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_generator,epochs=10,validation_data=val_generator,workers=10,use_multiprocessing=True)

In [ ]:
model.save('model_fine_tuned.pb')

# Evaluation on Test Set

In [ ]:
#indoor testset
model.evaluate(test_indoor_generator,use_multiprocessing=True,workers=10)

In [ ]:
#outdoor testset
model.evaluate(test_outdoor_generator,use_multiprocessing=True,workers=10)

In [ ]:
#belt testset
model.evaluate(test_belt_generator,use_multiprocessing=True,workers=10)